In [181]:
import pandas as pd
import numpy as np
import backtrader as bt
import datetime
import backtrader.feeds as btfeeds
from IPython.core.interactiveshell import InteractiveShell 
import pyecharts.options as opts
from pyecharts.charts import Grid,Line,Page,Scatter,Page,Bar
from pyecharts.faker import Faker
InteractiveShell.ast_node_interactivity = "all"
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller

CN = pd.read_excel('CN.xlsx',sheet_name = None)
data = pd.read_excel('idxdata.xlsx',sheet_name=None,skiprows=[0,1,2,3,4],index_col='Dates')

In [182]:
for k in CN:
    CN[k] = CN[k][['日期','开盘价(元)','收盘价(元)']]
    CN[k].columns = ['Dates','Open','Close']
    CN[k] = CN[k].set_index('Dates')
    temp = CN[k].between_time('9:45','15:00').resample('D').first()
    temp.loc[:,'Close'] = CN[k]['Close'].between_time('9:45','15:00').resample('D').last()
    CN[k] = temp.dropna()
    


In [183]:
wind_ticker = list(CN.keys())
bbg_ticker = list(data['XU1 Index'].FUT_CUR_GEN_TICKER.unique()[::-1])
i = 0
wtb = {}
btw = {}
for i in np.arange(0,27):
    wtb[wind_ticker[i]] = bbg_ticker[i]
    btw[bbg_ticker[i]] = wind_ticker[i]
for k in data:
    print(k)
    data[k].columns = ['Ticker','Open','Close']
    if k in ['SSE50 Index','SHSN300 Index','SH000905 Index','XIN9I Index','CNH Curncy']:
        data[k].Ticker = k.split()[0]
    data[k] = data[k].dropna()


XU1 Index
IFB1 Index
FFB1 Index
FFD1 Index
XIN9I Index
SHSN300 Index
SSE50 Index
SH000905 Index
CNH Curncy


In [ ]:
for idx,row in data['XU1 Index'].iterrows():
    print(idx)
    code = btw[row[0]]
    if idx in CN[code].index:
        data['XU1 Index'].loc[idx,'Close'] = CN[code].loc[idx,'Close']
        data['XU1 Index'].loc[idx,'Open'] = CN[code].loc[idx,'Open']


In [189]:
onshore_roll = data['FFD1 Index'].reset_index().groupby('Ticker').first().sort_values('Dates').Dates.to_list()
offshore_roll = data['XU1 Index'].reset_index().groupby('Ticker').first().sort_values('Dates').Dates.to_list()
all_roll = (onshore_roll + offshore_roll)
all_roll.sort()

In [198]:
cn_basis = data['XIN9I Index'][['Close','Open']]-data['XU1 Index'][['Close','Open']]
ih_basis = (data['SSE50 Index'][['Close','Open']]-data['FFB1 Index'][['Close','Open']])/data['SSE50 Index'][['Close','Open']]
spot_3ific = 3*data['SHSN300 Index'][['Close','Open']] - data['SH000905 Index'][['Close','Open']]
fut_3ific = 3*data['IFB1 Index'][['Close','Open']] - data['FFD1 Index'][['Close','Open']]
spot_2ific = 2*data['SHSN300 Index'][['Close','Open']] - data['SH000905 Index'][['Close','Open']]
fut_2ific = 2*data['IFB1 Index'][['Close','Open']] - data['FFD1 Index'][['Close','Open']]


basis_spread_ihcn = cn_basis - data['XIN9I Index'][['Close','Open']]*ih_basis
basis_spread_3ific = cn_basis - data['XIN9I Index'][['Close','Open']]*(spot_3ific - fut_3ific)/spot_3ific
basis_spread_2ific = cn_basis - data['XIN9I Index'][['Close','Open']]*(spot_2ific - fut_2ific)/spot_2ific
portvalue_ihcn = 10 * data['XIN9I Index'][['Close','Open']] * data['CNH Curncy'][['Close','Open']] - 300*data['FFB1 Index'][['Close','Open']]
portvalue_3ificcn =  ( data['XIN9I Index'][['Close','Open']] * data['CNH Curncy'][['Close','Open']] * 30
                    - data['IFB1 Index'][['Close','Open']] * 300 * 3 
                    + data['FFD1 Index'][['Close','Open']] * 200
                    )
portvalue_2ificcn =  ( data['XIN9I Index'][['Close','Open']] * data['CNH Curncy'][['Close','Open']] * 15
                    - data['IFB1 Index'][['Close','Open']] * 300 * 2 
                    + data['FFD1 Index'][['Close','Open']] * 200
                    )
basis_spread_ihcn_diff = basis_spread_ihcn.diff(1)
basis_spread_ihcn_diff.loc[all_roll,:] = np.nan
basis_spread_ihcn_intraday = basis_spread_ihcn.Close - basis_spread_ihcn.Open

basis_spread_3ific_diff = basis_spread_3ific.diff(1)
basis_spread_3ific_diff.loc[all_roll,:] = np.nan
basis_spread_3ific_intraday = basis_spread_3ific.Close - basis_spread_3ific.Open

basis_spread_2ific_diff = basis_spread_2ific.diff(1)
basis_spread_2ific_diff.loc[all_roll,:] = np.nan
basis_spread_2ific_intraday = basis_spread_2ific.Close - basis_spread_2ific.Open

portvalue_ihcn_diff = portvalue_ihcn.diff(1)
portvalue_ihcn_diff.loc[all_roll,:] = np.nan
portvalue_ihcn_intraday = portvalue_ihcn.Close - portvalue_ihcn.Open
portvalue_ihcn_intraday_adj = portvalue_ihcn_intraday - 10*basis_spread_ihcn_intraday*data['CNH Curncy'][['Close','Open']]
portvalue_ihcn_diff_adj = portvalue_ihcn_diff - 10*basis_spread_ihcn_diff*data['CNH Curncy'][['Close','Open']]

portvalue_3ificcn_diff = portvalue_3ificcn.diff(1)
portvalue_3ificcn_diff.loc[all_roll,:] = np.nan
portvalue_3ificcn_intraday = portvalue_3ificcn.Close - portvalue_3ificcn.Open
portvalue_ihcn_intraday_adj = portvalue_ihcn_intraday - 10*basis_spread_ihcn_intraday*data['CNH Curncy'][['Close','Open']]

portvalue_ihcn_diff_adj = portvalue_ihcn_diff - 30*basis_spread_ihcn_diff*data['CNH Curncy'][['Close','Open']]

portvalue_2ificcn_diff = portvalue_2ificcn.diff(1)
portvalue_2ificcn.loc[all_roll,:] = np.nan
portvalue_2ificcn_intraday = portvalue_2ificcn.Close - portvalue_2ificcn.Open
portvalue_ihcn_diff_adj = portvalue_ihcn_diff - 15*basis_spread_ihcn_diff*data['CNH Curncy'][['Close','Open']]







In [220]:
sc = (
     Scatter(
                init_opts=opts.InitOpts(
                width="540px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis(basis_spread_ihcn_diff['Close'])
        .add_yaxis(
                    series_name="Basis_Spread",
                    y_axis=portvalue_ihcn_diff_adj.Close.values.tolist(),
                    symbol_size = 5,
                    # color = 'blue',
                   
                    # is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    # linestyle_opts=opts.LineStyleOpts(width = 2)
                    )
        .set_global_opts(
                            title_opts = opts.TitleOpts(title='隔日收盘单笔持仓收益'),
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                type_ = 'value',
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Basis',
                                # max_ = 'dataMax',
                                # min_ = 'dataMin',
                                position = 'left',
                                is_scale = True,
                                min_interval = 1,
                                axislabel_opts=opts.LabelOpts(),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=False, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
)
sc.render('隔日收盘.html')

'c:\\Users\\Alienware\\Desktop\\ah\\股指\\sc.html'

In [ ]:
sc = (
     Scatter(
                init_opts=opts.InitOpts(
                width="540px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis(basis_spread_ihcn_intraday.values)
        .add_yaxis(
                    series_name="Basis_Spread",
                    y_axis=portvalue_ihcn_diff_adj.Close.values.tolist(),
                    symbol_size = 5,
                    # color = 'blue',
                   
                    # is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    # linestyle_opts=opts.LineStyleOpts(width = 2)
                    )
        .set_global_opts(
                            title_opts = opts.TitleOpts(title='隔日收盘单笔持仓收益'),
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                type_ = 'value',
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Basis',
                                # max_ = 'dataMax',
                                # min_ = 'dataMin',
                                position = 'left',
                                is_scale = True,
                                min_interval = 1,
                                axislabel_opts=opts.LabelOpts(),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=False, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
)
sc.render('sc.html')

In [199]:
totalPortValue = (
        Scatter(
                init_opts=opts.InitOpts(
                width="1440px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis(basis_spread_ihcn.index.date.tolist())
        .add_yaxis(
                    series_name="Basis_Spread",
                    y_axis=[opts.ScatterItem('ihcn',v, symbol_size = 7,itemstyle_opts = opts.ItemStyleOpts(color= '#6E6E6E' if v > 0 else '#642EFE'))
                        for v in basis_spread_ihcn_diff.Close ],
                    
                    color = 'black',
                   
                    # is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    # linestyle_opts=opts.LineStyleOpts(width = 2)
                    )
        .extend_axis(
                    yaxis=opts.AxisOpts(
                                name = 'PortValue',
                                position = 'left',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(),
                                splitline_opts=opts.SplitLineOpts(  is_show=False)
                            ),
                    )
        
        .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Basis',
                                # max_ = 'dataMax',
                                # min_ = 'dataMin',
                                position = 'right',
                                is_scale = True,
                                min_interval = 1,
                                axislabel_opts=opts.LabelOpts(),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=False, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
    )

totalCashUsage = (
    Bar()
    .add_xaxis(basis_spread_ihcn.index.date.tolist())
    .add_yaxis(
                series_name="ihcn",
                y_axis=[opts.BarItem('ihcn',v, itemstyle_opts = opts.ItemStyleOpts(color= 'red' if v > 0 else 'green'))
                        for v in portvalue_ihcn_diff_adj.Close.tolist() ],
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                #itemstyle_opts = opts.ItemStyleOpts(width=1.5,type_ = 'solid',opacity=0.73, )
    )
    
    .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'CNY',
                                is_scale = True,
                                # max_ = 200,
                                # min_ = 0,
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
    )
)
totalPortValue.overlap(totalCashUsage)
totalPortValue.render('res.html')

'c:\\Users\\Alienware\\Desktop\\ah\\股指\\res.html'